In [32]:
#https://huggingface.co/docs/datasets/v1.11.0/add_dataset.html

import pandas as pd
import re

# prepare dataset
df=pd.read_csv('../rajoy.csv')

# todo move this cleanup to the prepare-data
df=df[df.Text.str.len()>10]
df=df[~df.Text.str.contains("mero de expediente \d+/\d+", regex=True)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1896 entries, 1 to 2058
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    1896 non-null   object
 1   Text    1896 non-null   object
 2   Date    1896 non-null   int64 
 3   Term    1896 non-null   object
 4   Title   1205 non-null   object
dtypes: int64(1), object(4)
memory usage: 88.9+ KB


In [37]:
sentences=df["Text"].to_list()

# splits sentences using marks
split_stc=[re.split('(\.\s|\?|\!|;)',str(s)) for s in sentences]
stcs=[]
# joins with the mark again
for l in split_stc:
    liter=iter(l)
    stcs.append([h + next(liter,'') for h in liter])

# unflatten and remove sentences shorter than 10 chars
sentences=[n.strip() for s in stcs for n in s if len(n.strip())>10]
df=pd.DataFrame(sentences, columns=["Text"])

# Creating a dataframe with 50%
# values of original dataframe
part_55 = df.sample(frac = 0.55).reset_index()
 
# Creating dataframe with
# rest of the 50% values
rest_part_55 = df.drop(part_55.index).reset_index()

In [41]:
from datasets import load_dataset, Dataset,DatasetDict

ds_train=Dataset.from_pandas(part_55)
ds_test=Dataset.from_pandas(rest_part_55)
#ds=Dataset.from_dict({'text':sentences})
ds = DatasetDict()
ds['train'] = ds_train
ds['test'] = ds_test

ds_train.to_json('../rajoy_train.jsonl')
ds_test.to_json('../rajoy_test.jsonl')



Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

4121781

In [44]:
ds=load_dataset('.',data_files={'train': ['../rajoy_train.jsonl'],
                             'test':['../rajoy_test.jsonl']})

Using custom data configuration .-b1c978082bff5d3c


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
ds

DatasetDict({
    train: Dataset({
        features: ['index', 'Text'],
        num_rows: 28962
    })
    test: Dataset({
        features: ['index', 'Text'],
        num_rows: 23697
    })
})

In [28]:
# Copyright 2020 The HuggingFace Datasets Authors and the current dataset script contributor.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# TODO: Address all TODOs and remove all explanatory comments
"""The Spanish politician debates dataset"""


import csv
import json
import os

import datasets


# TODO: Add BibTeX citation
# Find for instance the citation on arxiv or on the dataset repo/website
_CITATION = """\
@InProceedings{huggingface:dataset,
title = {A Spanish politician debates dataset},
author={Juan Manuel Servera Bondroit},
year={2022}
}
"""

# TODO: Add description of the dataset here
# You can copy an official description
_DESCRIPTION = """\
This dataset is extracted from the www.congreso.es website, where you can download all the political debates for the last 10 terms in the form of text files.
"""

# TODO: Add a link to an official homepage for the dataset here
_HOMEPAGE = ""

# TODO: Add the licence for the dataset here if you can find it
_LICENSE = "Apache-2.0"

# TODO: Add link to the official dataset URLs here
# The HuggingFace Datasets library doesn't host the datasets but only points to the original files.
# This can be an arbitrary nested dict/list of URLs (see below in `_split_generators` method)
_URLS = {
    "first_domain": "https://huggingface.co/great-new-dataset-first_domain.zip",
    "second_domain": "https://huggingface.co/great-new-dataset-second_domain.zip",
}


# TODO: Name of the dataset usually matches the script name with CamelCase instead of snake_case
class SpanishDebatesDataset(datasets.GeneratorBasedBuilder):
    """The Spanish politician debates dataset."""

    VERSION = datasets.Version("0.1.0")

    # This is an example of a dataset with multiple configurations.
    # If you don't want/need to define several sub-sets in your dataset,
    # just remove the BUILDER_CONFIG_CLASS and the BUILDER_CONFIGS attributes.

    # If you need to make complex sub-parts in the datasets with configurable options
    # You can create your own builder configuration class to store attribute, inheriting from datasets.BuilderConfig
    # BUILDER_CONFIG_CLASS = MyBuilderConfig

    # You will be able to load one or the other configurations in the following list with
    # data = datasets.load_dataset('my_dataset', 'first_domain')
    # data = datasets.load_dataset('my_dataset', 'second_domain')
    # BUILDER_CONFIGS = [
    #     datasets.BuilderConfig(name="first_domain", version=VERSION, description="This part of my dataset covers a first domain"),
    #     datasets.BuilderConfig(name="second_domain", version=VERSION, description="This part of my dataset covers a second domain"),
    # ]

    # DEFAULT_CONFIG_NAME = "first_domain"  # It's not mandatory to have a default configuration. Just use one if it make sense.

    def _info(self):
        features = datasets.Features(
                {
                    "sentence": datasets.Value("string")
                }
        )
        # TODO: This method specifies the datasets.DatasetInfo object which contains informations and typings for the dataset
        # if self.config.name == "first_domain":  # This is the name of the configuration selected in BUILDER_CONFIGS above
            
        #     )
        # else:  # This is an example to show how to have different features for "first_domain" and "second_domain"
        #     features = datasets.Features(
        #         {
        #             "sentence": datasets.Value("string"),
        #             "option2": datasets.Value("string"),
        #             "second_domain_answer": datasets.Value("string")
        #             # These are the features of your dataset like images, labels ...
        #         }
        #     )
        return datasets.DatasetInfo(
            # This is the description that will appear on the datasets page.
            description=_DESCRIPTION,
            # This defines the different columns of the dataset and their types
            features=features,  # Here we define them above because they are different between the two configurations
            # If there's a common (input, target) tuple from the features, uncomment supervised_keys line below and
            # specify them. They'll be used if as_supervised=True in builder.as_dataset.
            # supervised_keys=("sentence", "label"),
            # Homepage of the dataset for documentation
            homepage=_HOMEPAGE,
            # License for the dataset if available
            license=_LICENSE,
            # Citation for the dataset
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        # TODO: This method is tasked with downloading/extracting the data and defining the splits depending on the configuration
        # If several configurations are possible (listed in BUILDER_CONFIGS), the configuration selected by the user is in self.config.name

        # dl_manager is a datasets.download.DownloadManager that can be used to download and extract URLS
        # It can accept any type or nested list/dict and will give back the same structure with the url replaced with path to local files.
        # By default the archives will be extracted and a path to a cached folder where they are extracted is returned instead of the archive
        urls = _URLS[self.config.name]
        data_dir = dl_manager.download_and_extract(urls)
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "train.jsonl"),
                    "split": "train",
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.VALIDATION,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "dev.jsonl"),
                    "split": "dev",
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.TEST,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "test.jsonl"),
                    "split": "test"
                },
            ),
        ]

    # method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
    def _generate_examples(self, filepath, split):
        # TODO: This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
        # The `key` is for legacy reasons (tfds) and is not important in itself, but must be unique for each example.
        with open(filepath, encoding="utf-8") as f:
            for key, row in enumerate(f):
                data = json.loads(row)
                yield key, {
                        "sentence": data["sentence"],
                    }
                # if self.config.name == "first_domain":
                #     # Yields examples as (key, example) tuples
                #     yield key, {
                #         "sentence": data["sentence"],
                #         "option1": data["option1"],
                #         "answer": "" if split == "test" else data["answer"],
                #     }
                # else:
                #     yield key, {
                #         "sentence": data["sentence"],
                #         "option2": data["option2"],
                #         "second_domain_answer": "" if split == "test" else data["second_domain_answer"],
                #     }